# Family-wise error rate (FWER): DSR, Bonferroni, Sidak, Holm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from functions import *

In [ ]:
FAST = True  # Set to False to have smoother / more stable plots

In [ ]:
import ray
ray.init()

In [ ]:
columns = ['1 - DSR', 'min p', 'p (Bonferroni)', 'p (Sidak)', 'p (Holm)']

T = 24
n_trials = 10

@ray.remote
def f( name, *, T = 24, n_trials = 10, SR = 0, N = 1000 if FAST else 10_000 ): 
    results = []
    for _ in range(N):
        X = generate_non_gaussian_data( T, n_trials, SR0=SR, name = name )
        r = deflated_sharpe_ratio( X, cluster = False, details = True )
        r['name'] = name
        r['T'] = T
        r['n_trials'] = n_trials
        r['SR'] = SR
        results.append(r)
    return pd.DataFrame( results )

all_results = { 
    (name, SR): f.remote( name, SR = SR, T = T, n_trials = n_trials)
    for name in ['gaussian', 'mild', 'moderate', 'severe']
    for SR in np.linspace( 0, 1, 21 )
}
all_results = { k: ray.get(v) for k,v in tqdm( all_results.items() )}

In [ ]:
for name in ['gaussian', 'mild', 'moderate', 'severe']:
    SR = 0
    results = all_results[name, SR]
    fig, axs = plt.subplots( 1, 5, figsize = (10,2.5), layout = 'constrained' )
    for ax, column in zip( axs, columns ):
        ax.hist( results[column], density = 'True', bins = 20, color = 'lightblue' )
        ax.set_ylim( 0, 5 )
        ax.set_yticks( [] )
        ax.set_xlim( 0, 1 )
        ax.set_title( column + "\n" + f"P[ p < .05 ] = {np.mean( results[column] < .05 ):.2f}" )
        ax.set_xlabel( "p-value" )
        for side in ['left', 'right', 'top']:
            ax.spines[side].set_visible(False)
    fig.suptitle( f"Distribution of p-values, distribution = {name}, T = {T}, n_trials = {n_trials}, SR = {SR}" )
    plt.show()